<center><h1><b>Write to MySQL Database From Google Sheets Using Python</b></h1></center>

<p>Required Library to be install:
<li>1. pip install gspread</li>
    <li>2. pip install oauth2client</li>
    <li>3. pip install mysql-connector</li>
Now Follow these steps to access Google Sheet via python :
    <li>1. Go to the <b>Google APIs Console</b>.</li>
    <li>2. Create a <b>new Project</b>.</li>
    <li>3. Click <b>Enable API</b>. Search for and enable <b>Google Drive API</b>.</li>
    <li>4. <b>Create Credentials</b> and Select <b>WebServer</b> and <b>Application Data</b>. In Project Role Select <b>Editor</b>.</li>
<li>5. Download JSON file save to your project folder and rename it as your required.</li>
    <li><b>Open the JSON file and copy the cilent_email address and Share your Google Sheet with that Email ID with Write Permisssion. Otherwise you will get error SheetsNotFound</b></li>
    <b>Note: When you run the script first time it gives error as to enable Google Sheet just click the link and its done. </b>
</p>

In [1]:
# import libraries
import gspread
import mysql_credential as key  # Your MySQl Credential python file here: mysql_credential.py
import mysql.connector
from oauth2client.service_account import ServiceAccountCredentials


In [2]:
# initialize variables for gspread
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scope)
client = gspread.authorize(creds)

In [3]:
# define method to pull data from spreadsheet
def GetSpreadsheetData(sheetName, worksheetIndex):    
    sheet = client.open(sheetName).get_worksheet(worksheetIndex)    
    return sheet.get_all_values()[1:]

In [4]:
# Client Email inside secret.json file must have edit permission on the Google Sheet you are using
data_product = GetSpreadsheetData('ProductsTest', 0)

In [5]:
# View Data From Google Sheets 
print(data_product[0])
print(len(data_product))

['Demo Product 1', 'Product Model 1', '98765987659801', 'Group 1', '1001']
110


In [6]:
# Client Email inside secret.json file must have edit permission on the Google Sheet you are using
data_group = GetSpreadsheetData('ProductsTest', 1)

In [7]:
# View Data From Google Sheets
data_group
print(data_group[0])
print(len(data_group))

['Group 1', 'Group Number 1', 'yes']
22


In [8]:
# method to write this data to MySQL

def ToMySQLTable(sql_data):
    try:
        connection = mysql.connector.connect(
            user = key.user,     
            password = key.password,        
            host = key.host,        
            database = key.database        
        )
        # Drop Group Table if it exists
        sql_drop = "DROP TABLE IF EXISTS {} ".format('GroupList')
        #create Group Table
        sql_create_table = """CREATE TABLE {}( 
            GroupId int NOT NULL AUTO_INCREMENT,
            GroupName VARCHAR(30) NOT NULL,
            GroupDescription VARCHAR(30),
            isActive VARCHAR(3) NOT NULL,
            PRIMARY KEY (GroupId)
            )""".format('GroupList')
         
        # Insert into GroupList Table From Google Sheet
        sql_insert_statement = """INSERT INTO {}( 
            GroupName,
            GroupDescription,
            isActive)
            VALUES ( %s,%s,%s )""".format('GroupList')
        
        cursor = connection.cursor()        
        cursor.execute(sql_drop)
        print('Table {} has been dropped'.format('GroupList'))        
        cursor.execute(sql_create_table)        
        print('Table {} has been created'.format('GroupList'))
        
        
        
        for i in sql_data:            
            cursor.execute(sql_insert_statement, i)
            
            connection.commit()        
           # print("Table {} successfully updated.".format('GroupList'))
            
        
        
            
    except mysql.connector.Error as error :        
        connection.rollback()        
        print("Error: {}. Table {} not updated!".format(error, 'GroupList'))
        
                
    finally:        
        cursor.execute('SELECT COUNT(*) FROM {}'.format('GroupList'))        
        rowCount = cursor.fetchone()[0]        
        print('GroupList', 'row count:', rowCount) 
        
        
        
        if connection.is_connected():            
            cursor.close()            
            connection.close()            
            print("MySQL connection is closed.")
                        

In [9]:
# method to write this data to MySQL

def ToMySQLTable1(sql_data):
    try:
        connection = mysql.connector.connect(
            user = key.user,     
            password = key.password,        
            host = key.host,        
            database = key.database        
        )
        # Drop Product Table if it exists
        sql_drop = "DROP TABLE IF EXISTS {} ".format('ProductList')
        # Create Product Table
        sql_create_table = """CREATE TABLE {}( 
            ProductId int NOT NULL AUTO_INCREMENT,
            ProductName varchar(255) NOT NULL,
            ProductModel varchar(255),
            ProductSerial varchar(255),
            GroupAssociated varchar(255) NOT NULL REFERENCES GroupList(GroupId),
            ProductMRP numeric(9,2),
            PRIMARY KEY (ProductId)
            )""".format('ProductList')
         
        # Insert into ProductList Table From Google Sheet
        sql_insert_statement = """INSERT INTO {}( 
            ProductName,
            ProductModel,
            ProductSerial,
            GroupAssociated,
            ProductMRP)
            VALUES ( %s,%s,%s,%s,%s )""".format('ProductList')
        cursor = connection.cursor()        
        cursor.execute(sql_drop)
        print('Table {} has been dropped'.format('ProductList'))        
        cursor.execute(sql_create_table)        
        print('Table {} has been created'.format('ProductList'))
        
        
        for i in sql_data:            
            cursor.execute(sql_insert_statement, i)
            
            connection.commit()        
            #print("Table {} successfully updated.".format('ProductList'))
        
        
            
    except mysql.connector.Error as error :        
        connection.rollback()        
        print("Error: {}. Table {} not updated!".format(error, 'ProductList'))
                
    finally:        
        
        cursor.execute('SELECT COUNT(*) FROM {}'.format('ProductList'))        
        rowCount = cursor.fetchone()[0]        
        print('ProductList', 'row count:', rowCount) 
        
        
        if connection.is_connected():            
            cursor.close()            
            connection.close()            
            print("MySQL connection is closed.")
                        

In [10]:
ToMySQLTable(data_group)

Table GroupList has been dropped
Table GroupList has been created
GroupList row count: 22
MySQL connection is closed.


In [11]:
ToMySQLTable1(data_product)

Table ProductList has been dropped
Table ProductList has been created
ProductList row count: 110
MySQL connection is closed.
